The first possible strategy for building a computer opponent is to make shots totally at random.

In [1]:
import random as pyrandom
from Gameboard import *

In [2]:
%%capture

# create Gameboard object
board = Gameboard()

First we write function to generate random board location.

In [3]:
def get_random_location():
    row_str = pyrandom.choice(Gameboard.row_labels)
    col_str = pyrandom.choice(Gameboard.col_labels)
    return row_str + str(col_str)

In [4]:
get_random_location()

'J7'

That's fine but we want to choose a random location from locations which are not aleady selected. For that, we create Random class. When it's instantiated, list of available locations is initialized. When a random location is chosen, it's taken out from available locations list.

In [5]:
class Random:
    def __init__(self):
        # possible locations list isn't affected by the methods
        self.possible_locations = []
        self.available_locations = []
        for i in Gameboard.row_labels:
            for j in Gameboard.col_labels:
                self.possible_locations.append(i + str(j))
                self.available_locations.append(i + str(j))
        
    def reset_available_locations(self):
        self.available_locations = self.possible_locations.copy()
        
    # chooses random location from available locations list and removes it if keep is False
    def choose(self, keep=False):
        choice = pyrandom.choice(self.available_locations)
        if not keep:
            self.available_locations.remove(choice)
        return choice
    
    # returns random location from possible locations list
    def choose_from_all(self):
        return pyrandom.choice(self.possible_locations)

In [6]:
# test Random class
random = Random()

for _ in range(95):
    random.choose()
    
random.available_locations

['A2', 'D4', 'D7', 'I2', 'J10']